# Descubra quem fez o ENEM 2016 apenas para treino

Neste desafio deverá descobrir quais estudantes estão fazendo a prova apenas para treino.

O contexto do desafio gira em torno dos resultados do ENEM 2016 (disponíveis no arquivo train.csv). Este arquivo, e apenas ele, deve ser utilizado para todos os desafios. Qualquer dúvida a respeito das colunas, consulte o [Dicionário dos Microdados do Enem 2016](https://s3-us-west-1.amazonaws.com/acceleration-assets-highway/data-science/dicionario-de-dados.zip).

Alguns estudantes decidem realizar prova do ENEM de forma precoce, como um teste (coluna IN_TREINEIRO). Neste desafio, você deve criar um modelo de classificação binária para inferir a mesma. Os resultados possíveis da sua resposta devem ser “0” ou “1”.

Salve sua resposta em um arquivo chamado answer.csv com duas colunas: `NU_INSCRICAO` e `IN_TREINEIRO`.

## Load libs

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Load dataset

In [2]:
traindf = pd.read_csv('train.csv', index_col=0)
traindf.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q041,Q042,Q043,Q044,Q045,Q046,Q047,Q048,Q049,Q050
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,5.0,A,A,A,A,A,A,A,B,D
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,NaN,A,A,C,A,B,A,A,C,A
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,A,A,A,A,C,A,A,B,A
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,5.0,C,A,A,A,A,D,A,A,A
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,A,A,A,A,A,A,A,A,A


In [3]:
traindf.shape

(13730, 166)

In [4]:
traindf.isnull().sum().sum()

253786

In [5]:
traindf.fillna(0, inplace=True)

In [6]:
traindf.isnull().sum().sum()

0

In [7]:
testdf = pd.read_csv('test.csv')
testdf.head()

,NU_INSCRICAO,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q006,Q024,Q025,Q026,Q027,Q047
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,43,RS,19,F,1,1,1,3,1,...,NaN,NaN,E,E,H,B,B,C,D,A
1,177f281c68fa032aedbd842a745da68490926cd2,15,PA,24,M,3,2,1,4,1,...,NaN,NaN,B,C,B,A,A,C,D,A
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,29,BA,16,F,2,1,3,0,1,...,NaN,NaN,E,F,G,B,B,A,NaN,D
3,5c356d810fa57671402502cd0933e5601a2ebf1e,41,PR,17,F,1,1,2,0,2,...,40.0,480.0,E,E,E,C,B,B,C,A
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,53,DF,19,F,1,1,1,1,1,...,80.0,720.0,E,E,E,B,B,B,D,A


In [8]:
testdf.shape

(4570, 43)

In [9]:
testdf.isnull().sum().sum()

20309

In [10]:
testdf.fillna(0, inplace=True)

In [11]:
testdf.isnull().sum().sum()

0

## Create answer df

In [12]:
answerdf = pd.DataFrame()

In [13]:
answerdf['NU_INSCRICAO'] = testdf['NU_INSCRICAO']
answerdf.head()

,NU_INSCRICAO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897
1,177f281c68fa032aedbd842a745da68490926cd2
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334
3,5c356d810fa57671402502cd0933e5601a2ebf1e
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3


In [14]:
answerdf.shape

(4570, 1)

## Data analysis

In [15]:
features = ['NU_IDADE', 'TP_ST_CONCLUSAO', 'TP_ANO_CONCLUIU']

## Train Test Split

Nesta etapa os dados serão separados da target feature, ou seja, aquela que se deseja prever, além de serem separados em dois conjuntos, um de treino e outro de teste.

Neste experimento os dados serão separados em uma proporção 80/20, ou seja, 80% das amostras do dataset serão selecionadas para o conjunto de treino e os outros 20% para o conjunto de teste, que será usado para validação do modelo.

### Separação das features do target

In [16]:
X = traindf[features]
X.head()

,NU_IDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU
1,24,1,4
2,17,2,0
3,21,3,0
4,25,1,9
5,28,1,4


In [17]:
y = traindf['IN_TREINEIRO']
y.head()

1    0
2    0
3    0
4    0
5    0
Name: IN_TREINEIRO, dtype: int64

### Separação dos dados em treino e teste

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Random Forest Classification

In [19]:
rfc = RandomForestClassifier(n_estimators=500)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
y_pred = rfc.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [21]:
f'acc: {(accuracy_score(y_test, y_pred) * 100).round(2)}%'

'acc: 99.75%'

In [22]:
y_pred = rfc.predict(testdf[features])
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [23]:
answerdf['IN_TREINEIRO'] = y_pred
answerdf.head()

,NU_INSCRICAO,IN_TREINEIRO
0,ba0cc30ba34e7a46764c09dfc38ed83d15828897,0
1,177f281c68fa032aedbd842a745da68490926cd2,0
2,6cf0d8b97597d7625cdedc7bdb6c0f052286c334,1
3,5c356d810fa57671402502cd0933e5601a2ebf1e,0
4,df47c07bd881c2db3f38c6048bf77c132ad0ceb3,0


In [24]:
testdf.shape, answerdf.shape

((4570, 43), (4570, 2))

In [25]:
answerdf.to_csv('answer.csv', index=False)